In [27]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import os.path
from os.path import join
import numpy as np
import imodelsx
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import data
import sys
files_dict = data.load_files_dict_single_site()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


success: 18
fail: ['Cincinnati', 'Columbus']


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [28]:
# site = 'Atlanta'
# site = 'Columbus'
site = 'WashingtonDC'
df = files_dict[site]
qs, responses_df, themes_df = data.split_single_site_df(df)

In [29]:
def numbered_list(responses):
    return '\n'.join([f'{i+1}. {c.strip()}' for i, c in enumerate(responses)])


themes_prompt = '''### You are given a question and a set of responses below.

**Question**: {question}

**Responses**:
{response_list}

### Group all responses into 2 or more non-overlapping themes.
### Return a comma-separated list, where each element is a theme, followed by the numbers of the responses that fall into that theme in brackets.
### **Example answer**: Theme 1: Negative responses [1, 2, 5], Theme 2: Positive responses [3, 4]

**Answer**: Theme 1:'''

llm = imodelsx.llm.get_llm('gpt-4', repeat_delay=3)

**Run single example**

In [30]:
# question, responses, theme_dict = data.get_data_for_question_single_site(
#     question_num=2, qs=qs, responses_df=responses_df, themes_df=themes_df)

# resps = responses[pd.notna(responses)]
# prompt = themes_prompt.format(
#     question=question,
#     response_list=numbered_list(resps)
# )
# print(prompt)
# llm(prompt)

### Screen valid questions
Valid questions have multiple unique responses.

In [31]:
def count_unique(resps):
    resps_match = resps.apply(str.lower)
    resps_match = resps_match.str.replace('[^\w\s]', '')
    # print(set(resps_match))
    return len(set(resps_match))


# screen valid questions
valid_question_nums = []
for question_num in tqdm(range(len(qs)), position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)
    resps = responses[pd.notna(responses)]

    # valid only if there are multiple unique responses
    if count_unique(resps) > 3:
        # print(resps)
        valid_question_nums.append(question_num)

    # print(np.unique(resps))
print('num valid qs', len(valid_question_nums), 'of', len(qs))

<>:3: DeprecationWarning: invalid escape sequence '\w'
<>:3: DeprecationWarning: invalid escape sequence '\w'
/tmp/ipykernel_1668665/1776519845.py:3: DeprecationWarning: invalid escape sequence '\w'
  resps_match = resps_match.str.replace('[^\w\s]', '')
100%|██████████| 46/46 [00:00<00:00, 917.86it/s]

num valid qs 28 of 46


### Run generating themes

In [32]:
themes_generated = {}
resps_list = {}
for question_num in tqdm(valid_question_nums, position=0):

    question, responses, theme_dict = data.get_data_for_question_single_site(
        question_num=question_num, qs=qs, responses_df=responses_df, themes_df=themes_df)
    resps = responses[pd.notna(responses)]

    prompt = themes_prompt.format(
        question=question,
        response_list=numbered_list(resps)
    )
    ans = llm(prompt)
    themes_generated[question_num] = ans
    resps.index = np.arange(len(resps)) + 1
    resps_list[question_num] = resps

  0%|          | 0/28 [00:00<?, ?it/s]

not cached


  4%|▎         | 1/28 [00:07<03:17,  7.31s/it]

not cached


  7%|▋         | 2/28 [00:15<03:24,  7.88s/it]

...

 93%|█████████▎| 26/28 [02:49<00:12,  6.42s/it]

not cached


 96%|█████████▋| 27/28 [02:57<00:06,  6.85s/it]

not cached


100%|██████████| 28/28 [03:02<00:00,  6.51s/it]


In [33]:
def dprint(*args, f):
    # print(*args, file=sys.stdout)
    print(*args, file=f)


with open(f'../figs/themes/themes_generated_{site}.md', 'w') as f:
    for question_num in valid_question_nums:
        dprint('### Question:', qs[question_num], f=f)
        dprint('\nResponses', f=f)
        for i in range(len(resps_list[question_num])):
            dprint(f'{i+1}. {resps_list[question_num].iloc[i]}', f=f)
        dprint('\nThemes', f=f)
        # print(themes_generated[question_num])
        themes = [s.strip(' ,:1234567890')
                  for s in themes_generated[question_num].split('Theme')]
        for i, theme in enumerate(themes):
            dprint(f'- Theme {i + 1}:', theme, f=f)
        dprint('', f=f)